In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
# Referência: https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html
from scipy import sparse
import time
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
%matplotlib inline
import os
from os import path
from wordcloud import WordCloud
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('all')

/home/emmanuel/.pyenv/versions/3.6.8/envs/cranium/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nl

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package unicode_samples is already up

True

In [2]:
start_time = time.time()


resenhas_imdb = pd.read_csv('../../dados_imdb/imdb-reviews-pt-br.csv')

'''
# PARA MOSTRAR TODOS
#https://stackoverflow.com/questions/62207066/pandas-does-not-show-the-complete-csv-file
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(resenhas_imdb)
'''

resenhas_imdb.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
resenhas_imdb['classificacao'] = resenhas_imdb['sentiment'].replace(['neg', 'pos'], [0,1])

In [4]:
def treinar_classificador(com_tratamento=True,
                          utiliza_stemmer=False,
                          utilizar_TFIDF=True,
                          utilizar_ngramas=True,
                          maximo_palavras=50):

    tamanho_texto_integral = len(resenhas_imdb['text_pt'])
    
    # https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    # https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
    regressao_logistica = LogisticRegression(solver='lbfgs', max_iter=200)


    if com_tratamento:

        palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
        from string import punctuation
        for pontuacao in punctuation:
            palavras_irrelevantes.append(pontuacao)

    else:
        palavras_irrelevantes = []

    frase_processada = list()

    # tokenizador = tokenize.WhitespaceTokenizer()
    tokenizador = tokenize.WordPunctTokenizer()
    stemmer = nltk.stem.RSLPStemmer()

    if utilizar_TFIDF:

        from sklearn.feature_extraction.text import TfidfVectorizer
        if utilizar_ngramas:
            tf_idf = TfidfVectorizer(lowercase=False, max_features=maximo_palavras, ngram_range=(1,2))
        else:
            tf_idf = TfidfVectorizer(lowercase=False, max_features=maximo_palavras)


    else:
        vetorizador = CountVectorizer(lowercase=False, max_features=maximo_palavras)



    # Retirada de acentos
    import unidecode

    jah_houve_tratamento_previo = 'texto_pt_tratado' in resenhas_imdb.columns

    if ((com_tratamento and not jah_houve_tratamento_previo) or com_tratamento == False):

        if not jah_houve_tratamento_previo:
           print('Necessário inicializar o tratamento no primeiro teste')

        for opiniao in resenhas_imdb.text_pt:

            palavras_texto = tokenizador.tokenize(opiniao)

            if com_tratamento:

                if utiliza_stemmer:
                    nova_frase = [stemmer.stem(unidecode.unidecode(palavra).lower()) for palavra in palavras_texto if
                              palavra not in palavras_irrelevantes]
                else:
                    nova_frase = [unidecode.unidecode(palavra).lower() for palavra in palavras_texto if
                                  palavra not in palavras_irrelevantes]
            else:
                nova_frase = [palavra for palavra in palavras_texto if palavra not in palavras_irrelevantes]

            frase_processada.append(' '.join(nova_frase))

    if com_tratamento:
        if not jah_houve_tratamento_previo:
            resenhas_imdb["texto_pt_tratado"] = frase_processada
        texto = 'texto_pt_tratado'

    else:
        texto = 'text_pt'

    if utilizar_TFIDF:
        matriz_treinamento = tf_idf.fit_transform(resenhas_imdb[texto])
    else:
        matriz_treinamento = vetorizador.fit_transform(resenhas_imdb[texto])

    '''
    matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words,
                                                      columns=vetorizador.get_feature_names())
    '''

    treino, teste, classe_treino, classe_teste = train_test_split(matriz_treinamento,
                                                                  resenhas_imdb.classificacao,
                                                                  random_state=42)


    regressao_logistica.fit(treino, classe_treino)
    acuracia = regressao_logistica.score(teste, classe_teste)

    if utilizar_TFIDF:
        pesos = pd.DataFrame(regressao_logistica.coef_[0].T,
                         index=tf_idf.get_feature_names())
    else:
        pesos = pd.DataFrame(regressao_logistica.coef_[0].T,
                             index=vetorizador.get_feature_names())

    print(pesos.nlargest(10, 0))
    print(pesos.nsmallest(10, 0))

    return acuracia

## Testando acurácia em diferentes configurações

In [5]:
def testar_configuracoes_treinamento(max_palavras=50):


    # https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
    print('Acurácia sem tratamento do texto e sem tf-idf: %.2f%%'
          % (treinar_classificador(com_tratamento=False,
                                   utilizar_TFIDF=False,
                                   maximo_palavras=max_palavras
                                   ) * 100))

    # https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
    print('Acurácia sem tratamento do texto e com tf_idf: %.2f%%'
          % (treinar_classificador(com_tratamento=False,
                                   utilizar_TFIDF=True,
                                   maximo_palavras=max_palavras
                                   ) * 100))
    # https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
    print('Acurácia com tratamento do texto e utilizaçao de TF-IDF SEM NGRAM(1,2): %.2f%%'
          % (treinar_classificador(com_tratamento=True,
                                   utiliza_stemmer=False,
                                   utilizar_TFIDF=True,
                                   utilizar_ngramas=False,
                                   maximo_palavras=max_palavras
                                   ) * 100))


    # https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
    # treinar_classificador(com_tratamento=True, utiliza_stemmer=False, utilizar_TFIDF=True, utilizar_ngramas=True)
    print('Acurácia com tratamento do texto e utilizaçao de TF-IDF COM NGRAM(1,2): %.2f%%'
          % (treinar_classificador(
                                    maximo_palavras=max_palavras

                                    ) * 100))


inicio_teste_1 = time.time()

testar_configuracoes_treinamento()

tempo_execucao_teste_1 = time.time() - inicio_teste_1
print('-----Tempo de execução do teste para 50 palavras no treinamento: %s segundos' % tempo_execucao_teste_1)

Necessário inicializar o tratamento no primeiro teste
                 0
também    0.324210
sua       0.171352
história  0.166779
dos       0.148691
ao        0.133008
mais      0.098034
Eu        0.093684
as        0.090859
como      0.087249
da        0.083078
               0
apenas -0.205901
não    -0.192853
esse   -0.154147
eles   -0.136159
isso   -0.128554
tão    -0.127347
ou     -0.124417
era    -0.121901
ser    -0.091349
está   -0.075213
Acurácia sem tratamento do texto e sem tf-idf: 65.83%
Necessário inicializar o tratamento no primeiro teste
                 0
sua       1.818516
dos       1.541166
história  1.343529
ao        1.254449
como      1.225772
Eu        1.122865
mais      1.012665
muito     0.968175
da        0.930276
as        0.877233
               0
não    -3.550051
apenas -2.033359
ou     -1.652949
isso   -1.652035
esse   -1.582702
eles   -1.556540
era    -1.499681
tão    -1.410298
ser    -1.244979
de     -1.154838
Acurácia sem tratamento do texto e com tf_idf:

In [6]:
inicio_teste_2 = time.time()

testar_configuracoes_treinamento(max_palavras=500)

tempo_execucao_teste_2 = time.time() - inicio_teste_2
print('-----Tempo de execução do teste para 500 palavras no treinamento: %s segundos' % tempo_execucao_teste_2)

                        0
excelente        1.129172
incrível         1.104655
ótimo            0.978070
brilhante        0.699586
definitivamente  0.698914
agradável        0.664628
hoje             0.610631
divertido        0.593059
gostei           0.545091
coração          0.535166
                 0
pior     -1.498643
terrível -1.345478
horrível -1.272499
chato    -1.176017
lixo     -0.884085
ruim     -0.856044
ruins    -0.793896
mal      -0.641306
roteiro  -0.490582
minutos  -0.478446
Acurácia sem tratamento do texto e sem tf-idf: 82.13%
                      0
excelente      7.262601
ótimo          6.865599
incrível       6.757264
um pouco       5.181909
hoje           4.114764
divertido      3.929144
melhores       3.709656
gostei         3.520210
sempre         2.970094
especialmente  2.938350
                  0
pior     -11.483275
horrível  -9.253768
ruim      -8.986648
terrível  -8.380280
mal       -5.357005
minutos   -4.666790
nada      -4.656812
dinheiro  -4.028547
roteiro

In [8]:
inicio_teste_3 = time.time()

testar_configuracoes_treinamento(max_palavras=1000)

tempo_execucao_teste_3 = time.time() - inicio_teste_3
print('-----Tempo de execução do teste para 1.000 palavras no treinamento: %s segundos' % tempo_execucao_teste_2)

/home/emmanuel/.pyenv/versions/3.6.8/envs/cranium/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


                      0
excelente      1.055918
incrível       1.035349
perfeito       1.017677
maravilhoso    0.957703
favorito       0.924862
ótimo          0.920105
perfeitamente  0.887125
recomendo      0.842150
maravilhosa    0.828679
obra           0.782877
                     0
piores       -1.826120
desperdício  -1.433111
pior         -1.403882
terrível     -1.252020
horrível     -1.237399
graça        -1.121724
chato        -1.036967
Infelizmente -0.926905
pobre        -0.910666
irritante    -0.877281
Acurácia sem tratamento do texto e sem tf-idf: 85.01%
                     0
excelente     7.057572
incrível      6.678979
ótimo         6.041985
perfeito      5.293061
um pouco      5.057858
dos melhores  4.999780
maravilhoso   4.872278
recomendo     4.623491
brilhante     4.356694
divertido     4.307817
                      0
pior         -10.886534
horrível      -8.820882
piores        -8.545649
ruim          -8.097495
terrível      -8.048497
chato         -6.423855
lixo    

In [7]:
tempo_execucao = time.time() - start_time
print('-----Tempo de execução: %s segundos' % tempo_execucao)

-----Tempo de execução: 169.85166478157043 segundos
